In [256]:
import requests
import pandas
import io
import datetime

In [257]:
station={'name': 'HAMBURG INNENSTADT', 'code': 'P0489'}
panel_size=40
panel_efficiency=0.2

In [258]:
url='https://opendata.dwd.de/weather/local_forecasts/poi/{}-MOSMIX.csv'.format(station['code'])
r=requests.get(url)

In [259]:
data=pandas.read_csv(io.StringIO(r.content.decode('utf-8')), skiprows=2, delimiter=';', decimal=",", na_values='---')
data['datetime']=[datetime.datetime.strptime('{} {}'.format(row['date'], row['time / content']), '%d.%m.%y %H:%M') for index, row in data.iterrows()]

In [260]:
startDate=datetime.datetime(2017, 9, 16, 5)
endDate=datetime.datetime(2017, 9, 23)

dates=pandas.DataFrame(columns=['datetime'])
currentDate=startDate
while currentDate <= endDate:
    dates=dates.append({'datetime': currentDate}, ignore_index=True)
    currentDate=currentDate+datetime.timedelta(hours=1)

In [261]:
fullData=pandas.merge(data, dates, how='outer', left_on='datetime', right_on='datetime')[['datetime', 'global radiation last hour']]
fullData=fullData.set_index('datetime')
fullData=fullData.sort_index()
fullData['global radiation last hour']=fullData['global radiation last hour'].fillna(0)
fullData['global radiation last hour']=fullData[['global radiation last hour']].rolling(window=7, center=True, min_periods=1).mean()
fullData['photo']=[row['global radiation last hour']*panel_size*panel_efficiency/3.6 for index, row in fullData.iterrows()]

In [262]:
fullData[['photo']].to_csv('photo.csv')